In [ ]:
# %% Deep learning - Section 15.148
#    Code challenge 23: identically random weights
#
#    1) Use PyTorch random seeds to create reproducible random networks
#    2) Create a network with an input layer (2 nodes), an hidden layer (8),
#       and an output layer (1); use nn.Sequential and not a Class (no need for
#       actual data or training)
#    3) Make 4 copies and use Xavier initialisation: i) no seed, ii) seed a,
#       iii) seed b, iv) seed a again
#    4) Networks ii and iv should be the same, print ii - iv to confirm
#    5) Plot the flattened weights to confirm

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [1]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F
import pandas              as pd
import scipy.stats         as stats
import sklearn.metrics     as skm
import time

from torch.utils.data                 import DataLoader,TensorDataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
plt.style.use('default')


In [46]:
# %% Build toy models

# Fully random
model_0 = nn.Sequential( nn.Linear(2,8),
                         nn.Linear(8,1),
                         nn.Linear(1,1) )

for p in model_0.named_parameters():
    if 'weight' in p[0]:
        nn.init.xavier_normal_(p[1].data)

# Seed a
torch.manual_seed(22)
model_1 = nn.Sequential( nn.Linear(2,8),
                         nn.Linear(8,1),
                         nn.Linear(1,1) )

for p in model_1.named_parameters():
    if 'weight' in p[0]:
        nn.init.xavier_normal_(p[1].data)

# Seed b
torch.manual_seed(88)
model_2 = nn.Sequential( nn.Linear(2,8),
                         nn.Linear(8,1),
                         nn.Linear(1,1) )

for p in model_2.named_parameters():
    if 'weight' in p[0]:
        nn.init.xavier_normal_(p[1].data)

# Seed a
torch.manual_seed(22)
model_3 = nn.Sequential( nn.Linear(2,8),
                         nn.Linear(8,1),
                         nn.Linear(1,1) )

for p in model_3.named_parameters():
    if 'weight' in p[0]:
        nn.init.xavier_normal_(p[1].data)


In [47]:
# %% Function to get the flattened weight vector

def flatten_weights(model):

    w = torch.cat([ param.detach().flatten()
                    for name, param in model.named_parameters()
                    if 'weight' in name ])

    return w


In [ ]:
# %% Extract weights

w0 = flatten_weights(model_0).numpy()
w1 = flatten_weights(model_1).numpy()
w2 = flatten_weights(model_2).numpy()
w3 = flatten_weights(model_3).numpy()

# Note that w1 - w3 should been zero
print(w0 - w1)
print(w1 - w3)
print(w1 - w2)


In [ ]:
# %% Plotting

phi = (1 + np.sqrt(5)) / 2
fig = plt.figure(figsize=(1.5*phi*5,5))

plt.plot(w0,'o',markersize=9,label='no seed')
plt.plot(w2,'^',markersize=9,label='seed b')
plt.plot(w1,'s',markersize=9,label='seed a')
plt.plot(w3,'+',markersize=9,label='seed a',markeredgewidth=3)

plt.axhline(y=0,color='grey',linestyle=':',linewidth=0.8)

plt.legend()
plt.xlabel('Vectorized weight index')
plt.ylabel('Weight value')
plt.title("Random (seeded) weights for four networks")

plt.savefig('figure37_code_challenge_23.png')
plt.show()
files.download('figure37_code_challenge_23.png')


In [ ]:
# %% Exercise 1
#    The code here only fixed the *weights*, not the biases. Does that mean that net_rs1a and net_rs1b are actually
#    DIFFERENT networks??

# Well, depends on the definition of "different network"; de facto, I'd say they
# may be considered the same starting network, because the purpose of the biases
# is only to allow for the error function to shift away from the origin. The of
# course fitting occurs and the weights will most likely change to diffent
# values, model fitting being inherently stochastic (even though they might well
# end up around the same minimum, depending on data, architecture,
# metaparameters, etc. etc.)


In [50]:
# %% Exercise 2
#    Change the code to plot/subtract the biases instead of the weights. Are the results what you expected? If not,
#    figure out why!

# Given that the seeding occurs before each model is created, also the biases
# are seeded in the same way

# Function
def flatten_biases(model):

    w = torch.cat([ param.detach().flatten()
                    for name, param in model.named_parameters()
                    if 'bias' in name ])

    return w


In [ ]:
# %% Exercise 2
#    Continue ...

# Get biases
b0 = flatten_biases(model_0).numpy()
b1 = flatten_biases(model_1).numpy()
b2 = flatten_biases(model_2).numpy()
b3 = flatten_biases(model_3).numpy()

# Plotting
phi = (1 + np.sqrt(5)) / 2
fig = plt.figure(figsize=(1.5*phi*5,5))

plt.plot(b0,'o',markersize=9,label='no seed')
plt.plot(b2,'^',markersize=9,label='seed b')
plt.plot(b1,'s',markersize=9,label='seed a')
plt.plot(b3,'+',markersize=9,label='seed a',markeredgewidth=3)

plt.axhline(y=0,color='grey',linestyle=':',linewidth=0.8)

plt.legend()
plt.xlabel('Vectorized weight index')
plt.ylabel('Weight value')
plt.title("Random (seeded) biases for four networks")

plt.savefig('figure38_code_challenge_23_extra2.png')
plt.show()
files.download('figure38_code_challenge_23_extra2.png')
